
# This is a simple example of an explicit use of the circulant matrix for discrete convolutions in probability

This exists as supporting code / an example for the Discrete Fourier Transform section contained in 

https://github.com/DerekB7/LinearAlgebra/blob/master/Vandermonde_Matrices_Permutations_and_Discrete_Fourier_Transform.ipynb


Note: there are numerous opportunities to further optimize this


In [1]:
import numpy as np
np.set_printoptions(precision = 2, linewidth=180)

# setup

# simple (PMF) distributions for some peculiar experiment,
# that can return a certain number of "heads"
# the number of heads = [0, 1, 2, 3, 4], 
# an associated PMFs for x and y

x = np.random.random(5)
x = x / x.sum() # normalize

y = np.random.random(5)
y = y / y.sum() # normalize

print("x vs y")
print( x, " vs ", y, "\n")

m = x.shape[0] * 2
z = np.zeros(m)
circulant_mat = np.zeros((m,m))

# direct convolution is done below, 
# and the circulant matrix is populated while doing this
for i in range(m):
    for idx in range(x.shape[0]):
        jdx = i - idx
        if jdx >= y.shape[0]  or jdx < 0:
            # simple setup with the non-negative distribution, though inefficient
            continue
        z[i] += x[idx] * y[jdx]
        circulant_mat[i,idx] = y[jdx]
  
padded_x = np.zeros(m)
# just some extra zeros in the padding to accomdoate 
# the higher order polynomial so to speak

for i in range(x.shape[0]):
    padded_x[i] += x[i]

# mathematically, the below has no impact 
# (as these entries get scaled by the padded zeros on padded_x), 
# but it finished off the circulant structure associated w/ our convolution
# It is important to know this exists

for row_idx in range(m):
    if np.isclose(circulant_mat[row_idx, 0], 0):
        continue
    else:
        for j in range(x.shape[0],m):
            circulant_mat[(row_idx + j) % m , j] = circulant_mat[row_idx, 0]

print(circulant_mat)

newz = circulant_mat @ padded_x 
# the alternative, direct, way of calculating z is via the use of this circulant matrix

print(" ")
print("# | probability    | difference between for loops and use of circulant matrix")
for i in range(m):
    print(i, "|", z[i], "|", z[i] - newz[i])

# of course there is lots of room to optimize calculations

x vs y
[ 0.28  0.01  0.06  0.39  0.26]  vs  [ 0.06  0.44  0.39  0.04  0.07] 

[[ 0.06  0.    0.    0.    0.    0.    0.07  0.04  0.39  0.44]
 [ 0.44  0.06  0.    0.    0.    0.    0.    0.07  0.04  0.39]
 [ 0.39  0.44  0.06  0.    0.    0.    0.    0.    0.07  0.04]
 [ 0.04  0.39  0.44  0.06  0.    0.    0.    0.    0.    0.07]
 [ 0.07  0.04  0.39  0.44  0.06  0.    0.    0.    0.    0.  ]
 [ 0.    0.07  0.04  0.39  0.44  0.06  0.    0.    0.    0.  ]
 [ 0.    0.    0.07  0.04  0.39  0.44  0.06  0.    0.    0.  ]
 [ 0.    0.    0.    0.07  0.04  0.39  0.44  0.06  0.    0.  ]
 [ 0.    0.    0.    0.    0.07  0.04  0.39  0.44  0.06  0.  ]
 [ 0.    0.    0.    0.    0.    0.07  0.04  0.39  0.44  0.06]]
 
# | probability    | difference between for loops and use of circulant matrix
0 | 0.0157246547069 | 0.0
1 | 0.123791541812 | 0.0
2 | 0.118182041414 | 0.0
3 | 0.0647568882156 | 0.0
4 | 0.2294436649 | 0.0
5 | 0.27158906305 | 0.0
6 | 0.122170109012 | 0.0
7 | 0.037217430673 | 0.0
8 | 0.017124